In [1]:
# Import Packages
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from datetime import datetime, date
# from mpl_toolkits.basemap import Basemap

In [2]:
# Change Directory
import os
os.chdir('swe_data/')
os.listdir()

['SN_SWE_WY1985.h5',
 'SN_SWE_WY1986.h5',
 'SN_SWE_WY1987.h5',
 'SN_SWE_WY1988.h5',
 'SN_SWE_WY1989.h5',
 'SN_SWE_WY1990.h5',
 'SN_SWE_WY1991.h5',
 'SN_SWE_WY1992.h5',
 'SN_SWE_WY1993.h5',
 'SN_SWE_WY1994.h5',
 'SN_SWE_WY1995.h5',
 'SN_SWE_WY1996.h5',
 'SN_SWE_WY1997.h5',
 'SN_SWE_WY1998.h5',
 'SN_SWE_WY1999.h5',
 'SN_SWE_WY2000.h5',
 'SN_SWE_WY2001.h5',
 'SN_SWE_WY2002.h5',
 'SN_SWE_WY2003.h5',
 'SN_SWE_WY2004.h5',
 'SN_SWE_WY2005.h5',
 'SN_SWE_WY2006.h5',
 'SN_SWE_WY2007.h5',
 'SN_SWE_WY2008.h5',
 'SN_SWE_WY2009.h5',
 'SN_SWE_WY2010.h5',
 'SN_SWE_WY2011.h5',
 'SN_SWE_WY2012.h5',
 'SN_SWE_WY2013.h5',
 'SN_SWE_WY2014.h5',
 'SN_SWE_WY2015.h5',
 'SN_SWE_WY2016.h5']

## Function Used in Later Code

In [3]:
### 2. Function for Extracting Index
def index_finder(lon,lat):
    # Longtitude finder
    if lon < -123.3 or lon > -117.6:
        print('Longitude of Input is out of range! lon:',lon)
        return None
    elif lat < 35.4 or lat > 42:
        print('Latitude of Input is out of range! lat:',lat)
    else: #longtitude and latitude are within reasonable range
        lon_idx = round((lon + 123.3) * 1000)
        lat_idx = round((lat - 35.4) * 1000)
    
        return lon_idx,lat_idx

## Read Data

In [86]:
# Discharge Data
gage = pd.read_csv('../gage_discharge_lat_lon.csv')
gage['swe'] = -1

In [87]:
gage.head()

,time,ft,m3,gage,lat,lon,swe
0,1984-10-01,54.0,1.529110,11402000,40.002947,-120.954399,-1
1,1984-10-02,52.0,1.472476,11402000,40.002947,-120.954399,-1
2,1984-10-03,49.0,1.387525,11402000,40.002947,-120.954399,-1
3,1984-10-04,49.0,1.387525,11402000,40.002947,-120.954399,-1
4,1984-10-05,48.0,1.359209,11402000,40.002947,-120.954399,-1


In [8]:
# Variable Defining
prev_year = 0

In [9]:
### 

for row_num in range(len(gage)):
    #if row_num == 0:
    row_data = gage.iloc[row_num,:]
    row_time = row_data['time']
    if row_time[0:4] == '2015' and row_data['gage']==11202710:
        break

In [10]:
'''
Gage with swe values: 11266500 
Gage with no swe values: 11402000, 11318500,11208000
Gage with certain swe values: 11185500
Gage with limited swe values: 11189500, 11202710
Note: Gage 11202710 starts from 1988 while others start from 1985
'''

'\nGage with swe values: 11266500 \nGage with no swe values: 11402000, 11318500,11208000\nGage with certain swe values: 11185500\nGage with limited swe values: 11189500, 11202710\nNote: Gage 11202710 starts from 1988 while others start from 1984\n'

In [11]:
row_data

time    2015-01-01
ft             6.6
m3        0.186891
gage      11202710
lat        36.1613
lon        -118.71
swe             -1
Name: 58530, dtype: object

In [12]:
# ---------- Inside For Loop -----------

In [13]:
### Extract Date from Gage Data to match SWE
date_format = "%Y-%m-%d"
d_date = datetime.strptime(row_time, date_format)

# Extract year of date
d_year = d_date.year
# Extract number of days from SWE Data
num_days = d_date- datetime.strptime('{}-1-1'.format(d_year),date_format)
num_days = num_days.days

print(f'Year: {d_year}  Day: {num_days}')


Year: 2015  Day: 0


In [14]:
## Obtain swe data
# if year of previous row does not match year of current row. Then Read Data
if prev_year != d_year: 
    swe = h5py.File(f'SN_SWE_WY{d_year}.h5', 'r')
    lat = swe['lat'][0][::-1]
    lon = swe['lon'][:,0]
    lats,lons = np.meshgrid(lat,lon)
prev_year = d_year

swe_data = swe['SWE'][num_days]
# flip over yaxis as lats are in a descending order --> need to change to ascending order
swe_data_flip = swe_data[:,::-1]

## Find SWE Interested Region with Lat Lon

In [16]:
lon_idx

4590.0

In [17]:
### 4. Find closest idx to the lower left & upper right corner
# ll_lon_idx,ll_lat_idx = index_finder(row_data['lon'],row_data['lat'])
# tr_lon_idx,tr_lat_idx = index_finder(day['tr_lon'],day['tr_lat'])
# region = swe_data_flip[ll_lon_idx:tr_lon_idx,ll_lat_idx:tr_lat_idx]

sur = 20

lon_idx,lat_idx = index_finder(row_data['lon'],row_data['lat'])
lon_idx,lat_idx = int(lon_idx),int(lat_idx)
region = swe_data_flip[lon_idx-sur:lon_idx+sur,lat_idx-sur:lat_idx+sur]




In [18]:
region

array([[-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       ...,
       [     0,      0, -32768, ..., -32768, -32768, -32768],
       [     0, -32768, -32768, ..., -32768, -32768, -32768],
       [     0, -32768, -32768, ..., -32768, -32768, -32768]], dtype=int16)

In [19]:
gage.iloc[:,6]

0       -1
1       -1
2       -1
3       -1
4       -1
        ..
83269   -1
83270   -1
83271   -1
83272   -1
83273   -1
Name: swe, Length: 83274, dtype: int64

In [176]:
### Obtain Value of Interested Region

# change null values to null
region=region.astype('float')
region[region == -32768] = np.nan

region_avg = np.nanmean(region)

if region_avg == np.nan:
    gage.iloc[row_num,6] = -1
else: # region avg is not null
    gage.iloc[row_num,6] = region_avg

In [178]:
gage.iloc[row_num,6]

0

In [44]:
####### RUN Data
row_time[0:4]

'1985'

# Data Joining Between Gage Time Series Data & SWE Value
- matching swe data to the gage time series data based on lat & lon of the gage
- **main code**

In [88]:
# gage = pd.read_csv('../gage_swe.csv')
date_format = "%Y-%m-%d"
sur = 20
index_list = []

### Run through all data
for ii,row_num in enumerate(range(len(gage))):
    
    ### Start from row xx
    if ii >=0:
    
        if ii % 100 == 0:
            print(f'-------- Processing Row Number {ii} out of {len(gage)} ---------')
        # Start with each row
        row_data = gage.iloc[row_num,:]
        row_time = row_data['time']

        if 1984 < int(row_time[0:4]) <2017  : # SWE has only data files from 1984 - 2016
            
            ### Obtain Value of Interested Region
            lon_idx,lat_idx = index_finder(row_data['lon'],row_data['lat'])
            lon_idx,lat_idx = int(lon_idx),int(lat_idx)
            
            if (lon_idx,lat_idx) not in index_list:

                ### Extract Date from Gage Data to match SWE
                d_date = datetime.strptime(row_time, date_format)

                # Extract year of date
                d_year = d_date.year
                # Extract number of days from SWE Data
                num_days = d_date- datetime.strptime('{}-1-1'.format(d_year),date_format)
                num_days = num_days.days

                ## Obtain swe data
                # if year of previous row does not match year of current row. Then Read Data
                if prev_year != d_year: 
                    swe = h5py.File(f'SN_SWE_WY{d_year}.h5', 'r')
                    lat = swe['lat'][0][::-1]
                    lon = swe['lon'][:,0]
                    lats,lons = np.meshgrid(lat,lon)
                prev_year = d_year

                swe_data = swe['SWE'][num_days]
                # flip over yaxis as lats are in a descending order --> need to change to ascending order
                swe_data_flip = swe_data[:,::-1]

                # get SWE values of surrounding region
                region = swe_data_flip[lon_idx-sur:lon_idx+sur,lat_idx-sur:lat_idx+sur]

                # change -32768 (null values) to null
                region=region.astype('float')
                region[region == -32768] = np.nan
                region_avg = np.nanmean(region)


                if pd.isna(region_avg): # if region_avg is null, change to -2
                    gage.iloc[row_num,6] = -2
                    index_list.append((lon_idx,lat_idx))
                else: # region avg is not null, assign the value with average
                    gage.iloc[row_num,6] = region_avg
            else: # (lon_idx,lat_idx) in index_list
                gage.iloc[row_num,6] = -2
    
        if ii % 1000 == 0:
            gage.to_csv('../gage_swe_1010.csv',index=False)
            print(f'---- Round {ii} Save to CSV file')

-------- Processing Row Number 0 out of 83274 ---------
---- Round 0 Save to CSV file


D:\Downloads\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: Mean of empty slice


-------- Processing Row Number 100 out of 83274 ---------
-------- Processing Row Number 200 out of 83274 ---------
-------- Processing Row Number 300 out of 83274 ---------
-------- Processing Row Number 400 out of 83274 ---------
-------- Processing Row Number 500 out of 83274 ---------
-------- Processing Row Number 600 out of 83274 ---------
-------- Processing Row Number 700 out of 83274 ---------
-------- Processing Row Number 800 out of 83274 ---------
-------- Processing Row Number 900 out of 83274 ---------
-------- Processing Row Number 1000 out of 83274 ---------
---- Round 1000 Save to CSV file
-------- Processing Row Number 1100 out of 83274 ---------
-------- Processing Row Number 1200 out of 83274 ---------
-------- Processing Row Number 1300 out of 83274 ---------
-------- Processing Row Number 1400 out of 83274 ---------
-------- Processing Row Number 1500 out of 83274 ---------
-------- Processing Row Number 1600 out of 83274 ---------
-------- Processing Row Number 1

---- Round 14000 Save to CSV file
-------- Processing Row Number 14100 out of 83274 ---------
-------- Processing Row Number 14200 out of 83274 ---------
-------- Processing Row Number 14300 out of 83274 ---------
-------- Processing Row Number 14400 out of 83274 ---------
-------- Processing Row Number 14500 out of 83274 ---------
-------- Processing Row Number 14600 out of 83274 ---------
-------- Processing Row Number 14700 out of 83274 ---------
-------- Processing Row Number 14800 out of 83274 ---------
-------- Processing Row Number 14900 out of 83274 ---------
-------- Processing Row Number 15000 out of 83274 ---------
---- Round 15000 Save to CSV file
-------- Processing Row Number 15100 out of 83274 ---------
-------- Processing Row Number 15200 out of 83274 ---------
-------- Processing Row Number 15300 out of 83274 ---------
-------- Processing Row Number 15400 out of 83274 ---------
-------- Processing Row Number 15500 out of 83274 ---------
-------- Processing Row Number 1

KeyboardInterrupt: 

In [ ]:
# save gage file to csv
gage.to_csv('../gage_swe.csv',index=False)

In [81]:
gage.iloc[80000:,6].value_counts()

 0.000    2615
-1.000     638
 1.000       9
 0.500       3
 0.625       2
 0.750       2
 1.250       1
 1.750       1
 1.500       1
 4.250       1
 0.875       1
Name: swe, dtype: int64

In [42]:
'''
## SWE value indication
Initial Assigned Value: -1
N/A value: -2
Rest are normal values
'''

-1.0

In [ ]:
###############

In [69]:
gage['swe'].nunique()

11672

In [ ]:
# ---------------------------

In [ ]:
# # Read Gage Lat Lon Data
# tar_gage = pd.read_csv('target_gages.csv')
# tar_gage

# # ----------- Finish For Loop -----------
# # Convert to DataFrame
# re = pd.DataFrame(region)
# # re.index = lon[ll_lon_idx:tr_lon_idx]
# # re.columns = lat[ll_lat_idx:tr_lat_idx]
# print(re)